# SBS 평가: Denoising (학습 모델) + Deconvolution (최소제곱법)

이 노트북은 Step-by-Step 파이프라인의 중간 성능을 확인하기 위해 제작되었습니다.
- **1단계 (Denoising)**: 학습된 DnCNN 모델을 사용하여 노이즈를 제거합니다.
- **2단계 (Deconvolution)**: 고전적인 이미지 복원 기법인 최소제곱법(Least Squares)을 사용하여 Deconvolution을 수행합니다.

이를 통해 전체 SBS 파이프라인(DnCNN + Unet)의 학습이 완료되기 전에 Denoising 모델의 성능과 한계를 잠정적으로 평가할 수 있습니다.


In [ ]:
# @title 1. 환경 설정
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 프로젝트 폴더 경로 설정 (본인의 환경에 맞게 수정)
import os
PROJECT_PATH = "/content/drive/MyDrive/Data Scientist/Project/Week5/week5"
os.chdir(PROJECT_PATH)

# 시스템 경로에 프로젝트 루트 추가
import sys
sys.path.append(PROJECT_PATH)

print(f"Current working directory: {os.getcwd()}")


In [ ]:
# @title (Optional) 1-1. Install Dependencies
# @markdown `loguru` 라이브러리가 설치되어 있지 않은 경우에만 이 셀을 실행하세요.
%pip install loguru --quiet


In [ ]:
# @title 2. 설정값 정의
# @markdown ---
# @markdown ### **1. 필수 경로 설정**
# @markdown Denoising 모델의 체크포인트 파일 경로를 지정하세요.
DENOISING_CKPT_PATH = "logs_sbs_denoising_dncnn/00001_train/checkpoints/checkpoint_best.ckpt" # @param {type:"string"}
# @markdown ---
# @markdown ### **2. 데이터 및 결과 폴더 설정**
# @markdown 테스트 데이터셋과 결과 파일을 저장할 폴더를 지정하세요.
TEST_DATA_PATH = "dataset/test_y" # @param {type:"string"}
RESULT_DIR = "result_sbs_denoising_ls_joint" # @param {type:"string"}
# @markdown ---
# @markdown ### **3. 공동 복원 하이퍼파라미터**
# @markdown Regularization 강도 (0에 가까울수록 강한 복원, 클수록 노이즈 억제)
LAMBDA = 1e-3 # @param {type:"number"}
# @markdown ---

print(f"✅ Joint Deconvolution (All Angles) Mode Enabled.")
print(f"Denoising Checkpoint: {DENOISING_CKPT_PATH}")
print(f"Test Data: {TEST_DATA_PATH}")
print(f"Result Directory: {RESULT_DIR} (Joint)")
print(f"Deconvolution Lambda: {LAMBDA}")


In [ ]:
# @title 3. 다각도 공동 복원(Joint Deconvolution) 함수 구현
import torch
import torch.fft as fft
from dataset.forward_simulator import ForwardSimulator, dipole_kernel
from params import config as global_config

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모든 방향에 대한 Dipole Kernel을 k-space 상에서 미리 생성
dipole_kernels_k = [
    dipole_kernel(matrix_size=global_config.image_size, B0_dir=direction).to(device)
    for direction in global_config.conv_directions
]
print(f"✅ Successfully created {len(dipole_kernels_k)} dipole kernels for joint deconvolution.")


def joint_deconv_all_angles(denoised_tensor: torch.Tensor, kernels_k: list[torch.Tensor], lambda_reg: float) -> torch.Tensor:
    """
    Performs joint deconvolution using all available angles (directions).
    Implements the formula: X_hat = sum(H_conj * Y) / (sum(|H|^2) + lambda)
    """
    # 입력 이미지를 k-space로 변환
    denoised_k = fft.fftn(denoised_tensor, dim=(-2, -1))

    # 분자(numerator)와 분모(denominator)의 합을 저장할 텐서 초기화
    # torch.zeros_like()를 사용하여 동일한 shape과 device를 갖도록 함
    numerator_sum = torch.zeros_like(denoised_k)
    denominator_sum = torch.zeros_like(torch.abs(kernels_k[0])**2)
    
    # 각 각도(theta)에 대해 분자/분모를 누적
    for kernel_k in kernels_k:
        numerator_sum += torch.conj(kernel_k) * denoised_k
        denominator_sum += torch.abs(kernel_k)**2
        
    # 최종 수식 적용
    denominator_sum += lambda_reg
    
    # 0으로 나누는 것을 방지
    denominator_sum[denominator_sum == 0] = 1.0
    
    deconv_k = numerator_sum / denominator_sum
    
    # 다시 이미지 공간으로 변환
    deconv_image = fft.ifftn(deconv_k, dim=(-2, -1))
    
    # 실수부만 반환
    return deconv_image.real

print("Joint deconvolution function is ready.")


In [ ]:
# @title 4. Denoising 모델 로드
from pathlib import Path
from code_denoising.core_funcs import get_model, ModelType
from params import config as global_config, dncnnconfig # 💡 global_config 추가

print(f"Loading Denoising checkpoint from: {DENOISING_CKPT_PATH}")

try:
    denoising_ckpt_path = Path(DENOISING_CKPT_PATH)
    if not denoising_ckpt_path.exists():
        raise FileNotFoundError(f"Denoising checkpoint not found at {denoising_ckpt_path}")

    denoising_checkpoint = torch.load(denoising_ckpt_path, map_location=device)
    
    # 💡 --- 모델 설정 복원 로직 (create_evaluation_results.py와 동일하게) ---
    # 1. 체크포인트에서 모델 타입 및 설정을 가져옴
    model_type_from_ckpt = denoising_checkpoint.get("model_type", "dncnn") # 기본값 dncnn
    model_config_from_ckpt = denoising_checkpoint.get('model_config')
    
    if model_config_from_ckpt:
        print("Found model_config in checkpoint. Using it to build the model.")
        model_config = model_config_from_ckpt
    else:
        # 2. model_config가 없는 구형 체크포인트의 경우, 기본 dncnnconfig 사용
        print("model_config not found in checkpoint. Falling back to default dncnnconfig.")
        model_config = dncnnconfig
    
    # 모델 생성 및 state_dict 로드
    denoising_network = get_model(model_config, model_type_from_ckpt).to(device)
    denoising_network.load_state_dict(denoising_checkpoint['model_state_dict'])
    denoising_network.eval()
    
    print(f"Successfully loaded Denoising model ({model_type_from_ckpt}).")

except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    denoising_network = None


In [ ]:
# @title 5. 파이프라인 실행 및 결과 저장
import shutil
from tqdm.notebook import tqdm
import numpy as np
from torch.utils.data import DataLoader
from code_denoising.datawrapper.datawrapper import DataKey, get_data_wrapper_loader, LoaderConfig

if denoising_network:
    # --- 기존 결과 폴더 삭제 ---
    result_path = Path(RESULT_DIR)
    if result_path.exists():
        print(f"Removing old '{RESULT_DIR}' directory...")
        shutil.rmtree(result_path)
    result_path.mkdir(parents=True, exist_ok=True)
    
    # --- 데이터 로더 설정 ---
    loader_cfg: LoaderConfig = {
        "data_type": global_config.data_type,
        "batch": 8, # GPU 메모리에 맞춰 배치 크기 조절 가능
        "num_workers": 2,
        "shuffle": False,
        "augmentation_mode": 'none',
        "training_phase": 'end_to_end',
        "noise_type": global_config.noise_type,
        "noise_levels": global_config.noise_levels,
        "conv_directions": global_config.conv_directions
    }
    data_loader, _ = get_data_wrapper_loader(
        file_path=[TEST_DATA_PATH],
        training_mode=False,
        data_wrapper_class='controlled',
        **loader_cfg
    )

    if not data_loader:
        print(f"Failed to create data loader from {TEST_DATA_PATH}. No data found?")
    else:
        print(f"\n[Phase 1/1] Creating result files from Denoising + Joint Deconvolution pipeline...")
        with torch.no_grad():
            for data in tqdm(data_loader, desc="Processing images"):
                image_noise = data[DataKey.image_noise].to(device)
                filenames = data[DataKey.name]

                # Step 1: Denoising (학습 모델)
                denoised_image_batch = denoising_network(image_noise)

                # Step 2: Joint Deconvolution (다각도 공동 복원)
                # 배치 내 각 이미지에 대해 개별적으로 복원 수행
                for i in range(denoised_image_batch.shape[0]):
                    single_denoised_tensor = denoised_image_batch[i:i+1]
                    
                    # 새로 구현된 공동 복원 함수 호출
                    final_image_tensor = joint_deconv_all_angles(
                        single_denoised_tensor, 
                        dipole_kernels_k, 
                        LAMBDA
                    )

                    # 결과 저장
                    pred_np = final_image_tensor.squeeze().cpu().numpy()
                    base_filename = Path(filenames[i]).stem
                    np.save(result_path / f"{base_filename}.npy", pred_np)
        
        print(f"\nFinished! Results are saved in '{RESULT_DIR}'.")
        print("You can now run the 'evaluate.ipynb' notebook to calculate the scores.")
else:
    print("\nSkipping pipeline execution because the model failed to load.")

